# LuxendoManager Example Notebook

In [ ]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [ ]:
# --- Set input data path ---
main_path = "../../pymif_test_data/luxendo"
dataset_name = "3T_2C_2895Z_2076Y_2860Z"

# main_path = "/g/mif/common/Data/Z_from_LCS_24-07-16/Coll_Montse/MouseEmbryo_micemetyleneBlue/2022-08-09_132304/processed"
# dataset_name = "20221108-162726_Task_2_LM_FullRes_C"

# main_path = "/mif-users/Users/Arias_Adrian/scans_CSL_P21/2400_31573_s"
# dataset_name = "20250523-105843_Task_1_stitching_1_C"

luxendo_path = f"{main_path}/{dataset_name}"

In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
dataset = mm.LuxendoManager(path=luxendo_path, chunks=(1,1,8,3000,3000))

In [ ]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

In [ ]:
dataset.data[0]

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster to open but not interactive.
# NOT RECOMMENDED FOR LARGE h5 DATASETS!
# To visualize lower resolution, use the next cell.
# To visualize the whole dataset at full resolution, convert data into ome-zarr

start = time.time()
viewer = dataset.visualize()
print(f"{time.time()-start}")

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# The start_level-stop_level options, generate lower resolution layers and load them into memory
# NOTE: This works because the dataset has 5 resolution layers
# It takes some time because it has to open the files and read the corresponding layers
# Best would be to resave the dataset in ome-zarr (see next cell on how to do it)

start = time.time()
viewer = dataset.visualize( 
                        start_level = 5, 
                        stop_level = 6, 
                        in_memory = True 
                    )
print(f"{time.time()-start}")

In [ ]:
# --- Build pyramid if not already ---
dataset.build_pyramid(
                    num_levels=8, 
                    downscale_factor=2,
                    start_level=0
                    )

# --- Optional: Write to OME-Zarr format ---
output_zarr_path = f"{main_path}/{dataset_name}.zarr"
dataset.write(output_zarr_path)

In [ ]:
# --- Show metadata summary for updated dataset ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

In [ ]:
dataset.close()

# Now we use the ZarrManager to load the new data and display it

In [ ]:
input_zarr_path = output_zarr_path

In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
# Here, we use the same chunks.
zarr_dataset = mm.ZarrManager(path=input_zarr_path)#, chunks=(1,1,32,2048,2048))

In [ ]:
for i in zarr_dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")
print("CHUNK SIZE:", zarr_dataset.chunks)

In [ ]:
zarr_dataset.data[0]

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster but not interactive.
start = time.time()
viewer = zarr_dataset.visualize()
print(f"{time.time()-start}")

In [ ]:
start = time.time()
# zarr_dataset.data = [level.rechunk((1, 1, 32, 512, 512)) for level in zarr_dataset.data]
viewer = zarr_dataset.visualize(                        
                        start_level = 0, 
                        stop_level = 2, 
                        in_memory = False 
)
print(f"{time.time()-start}")